In [38]:
# imports 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 1. British Windfarm

### 1.1. Load Datasets

In [39]:
# load data into pandas dataframe
turbine_one = pd.read_csv('../data/Kelmarsh_SCADA_2021_3087/Turbine_Data_Kelmarsh_1_2021-01-01_-_2021-07-01_228.csv', header=9)
turbine_two = pd.read_csv('../data/Kelmarsh_SCADA_2021_3087/Turbine_Data_Kelmarsh_2_2021-01-01_-_2021-07-01_229.csv', header=9)
turbine_three = pd.read_csv('../data/Kelmarsh_SCADA_2021_3087/Turbine_Data_Kelmarsh_3_2021-01-01_-_2021-07-01_230.csv', header=9)
turbine_four = pd.read_csv('../data/Kelmarsh_SCADA_2021_3087/Turbine_Data_Kelmarsh_4_2021-01-01_-_2021-07-01_231.csv', header=9)
turbine_five = pd.read_csv('../data/Kelmarsh_SCADA_2021_3087/Turbine_Data_Kelmarsh_5_2021-01-01_-_2021-07-01_232.csv', header=9)
turbine_six = pd.read_csv('../data/Kelmarsh_SCADA_2021_3087/Turbine_Data_Kelmarsh_6_2021-01-01_-_2021-07-01_233.csv', header=9)

In [40]:
# Extract only the first 28 columns, which contain the data we need
turbine_one = turbine_one.iloc[:, :28]
turbine_two = turbine_two.iloc[:, :28]
turbine_three = turbine_three.iloc[:, :28]
turbine_four = turbine_four.iloc[:, :28]
turbine_five = turbine_five.iloc[:, :28]
turbine_six = turbine_six.iloc[:, :28]

# since we have to perform the same operations on each turbine, we can create a list of the dataframes and iterate through them
turbine_list = [turbine_one, turbine_two, turbine_three, turbine_four, turbine_five, turbine_six]

# print the keys of the first turbine to see what columns we have
# print(turbine_one.keys())

# rename the '# Date and time' column to 'Date' for easier access, convert it to pd.datetime and set it as the index
for i, turbine in enumerate(turbine_list):
    turbine.rename(columns={'# Date and time': 'Datetime'}, inplace=True)
    turbine['Datetime'] = pd.to_datetime(turbine['Datetime'])
    turbine.set_index('Datetime', inplace=True)
    print(f"There exist {len(turbine)} data points for turbine {i+1}")

There exist 26064 data points for turbine 1
There exist 26064 data points for turbine 2
There exist 26064 data points for turbine 3
There exist 26064 data points for turbine 4
There exist 26064 data points for turbine 5
There exist 26064 data points for turbine 6


### 1.2. Cleaning Data

In [41]:
# delete all rows that contain NaN values
for i, turbine in enumerate(turbine_list):
    turbine.dropna(inplace=True)
    print(f"There exist {len(turbine)} data points for turbine {i+1} after removing NaN values")

There exist 23275 data points for turbine 1 after removing NaN values
There exist 23229 data points for turbine 2 after removing NaN values
There exist 23236 data points for turbine 3 after removing NaN values
There exist 23188 data points for turbine 4 after removing NaN values
There exist 23177 data points for turbine 5 after removing NaN values
There exist 23324 data points for turbine 6 after removing NaN values


In [42]:
# check for duplicates
for i, turbine in enumerate(turbine_list):
    print(f"There exist {len(turbine[turbine.duplicated()])} duplicated rows in turbine {i+1}")

There exist 0 duplicated rows in turbine 1
There exist 0 duplicated rows in turbine 2
There exist 0 duplicated rows in turbine 3
There exist 0 duplicated rows in turbine 4
There exist 0 duplicated rows in turbine 5
There exist 0 duplicated rows in turbine 6


### 1.3. Plot relevant Variables